# Case 4 - Machine learning uitvoering

### Inladen Packages

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier

### Inladen data

In [3]:
Train = pd.read_csv('Train_Acitivity_Tracker_Labeled.csv')
Test = pd.read_csv('Test.csv')

In [4]:
Columns = ['0','1','2','3','4','5']
LabelsColumn = ['0','1','2','3','4','5','Group']

## Uitvoeren Machine learning models

In [5]:
TrainOrTest = 0

In [6]:
def Get_Model(Model):
    if Model == 'DT':
        return DecisionTreeClassifier()
    elif Model == 'KN':
        return RandomForestClassifier(n_estimators=11,random_state=True)
    elif Model == 'RF':
            return KNeighborsClassifier(n_neighbors=7)

In [7]:
def ML(X_test,X_train,y_test,y_train,modelType):
    modelML = Get_Model(modelType)
    modelML.fit(X_train,y_train.values.ravel())
    pred_ML = modelML.predict(X_test)
    if TrainOrTest == 0:
        scoreML = accuracy_score(y_test,pred_ML)
        scoreML = scoreML.round(4)
        print("Score is:" ,scoreML*100,'%')
    return pred_ML, X_test

### Extra Features

In [8]:
def rolling(label):
    OR = Train[Train['labels']== label]
    LB = OR[['0','1','2']].rolling(window= 50).mean()
    #LB = LB[['0','1','2']].shift(-2, axis =0)
    LB = LB.dropna(axis=0)
    LB = LB.assign(labels = label)
    return LB

In [9]:
lb1= rolling(1)
lb2= rolling(2)
lb3= rolling(3)
lb4= rolling(4)
lb5= rolling(5)
lb6= rolling(6)
lb7= rolling(7)
lb8= rolling(8)
lb9= rolling(9)
lb10= rolling(10)
lb11= rolling(11)
lb12= rolling(12)
Train =  pd.concat([lb1,lb2,lb3,lb4,lb5,lb6,lb7,lb8,lb9,lb10,lb11,lb12], ignore_index=False)

In [10]:
Train['3'] = (Train['0'] + Train['1'] + Train['2']).mean()
Train['4'] = np.std(Train['0'] + Train['1'] + Train['2'])
Train['5'] = (Train['0'] + Train['1'] + Train['2']) ** 2

Test['3'] = (Test['0'] + Test['1'] + Test['2']).mean()
Test['4'] = np.std(Test['0'] + Test['1'] + Test['2'])
Test['5'] = (Test['0'] + Test['1'] + Test['2']) ** 2

### Guess Group

Grouping zorgt voor hogere accuracy met de train dataset maar bij kaggle een lagere score

#### Create Groups

<table>
    <tr>
        <th>101 Bewegen:</th>
        <th>102 Stil:</th>
        <th>103 Transitie:</th>
    </tr>
    <tr>
        <td>1 Lopen</td>
        <td>4 Zitten</td>
        <td>7 Staan_Naar_Zitten</td>
    </tr>
    <tr>
        <td>2 Lopen_Trap_op</td>
        <td>5 Staan</td>
        <td>8 Zitten_Naar_Staan</td>
    </tr>
    <tr>
        <td>3 Lopen_Trap_af</td>
        <td>6 Liggen</td>
        <td>9 Zitten_Naar_Liggen</td>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td>10 Liggen_Naar_Zitten</td>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td>11 Staan_Naar_Liggen</td>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td>12 Liggen_Naar_Staan</td>
    </tr>
</table>

In [11]:
Check = Train
Con = [
    (Train['labels'] < 4),
    (Train['labels'] < 7) & (Train['labels'] > 3)
]
Choices = [101,102]
Check['Group'] = np.select(Con,Choices,default=103)

#### Machine Learning Groups

Give Test dataframe groups using Train dataframe

In [12]:
if TrainOrTest == 1:
    X_train = Check[Columns]
    y_train = Check[['Group']]
    X_test = Test[Columns]
    y_test = ''
else:
    X = Train[Columns]
    y = Train[['Group']]
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

#### Predict

In [13]:
if TrainOrTest == 0:
    Prediction, X_test = ML(X_test,X_train,y_test,y_train,'RF')

### Guess Label

#### Create X & y

In [14]:
if TrainOrTest == 1:
    X_test = pd.DataFrame(X_test)
    X_test['Group'] = Prediction
    X_train = Train[LabelsColumn]
    y_train = Train[['labels']]
    y_test = ''
else:
    X = Train[LabelsColumn]
    y = Train[['labels']]
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

#### Predict

In [15]:
Prediction, X_test = ML(X_test,X_train,y_test,y_train,'DT')

Export

In [16]:
if TrainOrTest == 1:
    Export = pd.DataFrame(Test['Index'])
    Export['labels'] = Prediction
    Export.to_csv('Kaggle.CSV',index=False)